In [1317]:
import numpy as np
from sklearn import preprocessing
import os.path
import librosa
import warnings
import scipy.stats as stats
import scipy as sp
from sklearn.preprocessing import MinMaxScaler
from scipy import spatial

#### **2. Extracção de Features.**
2.1. Processar as features do ficheiro top100_features.csv.

2.1.1. Ler o ficheiro e criar um array numpy com as features disponibilizadas.

In [1318]:
def extract_features():
    data = np.genfromtxt("Features/top100_features.csv", delimiter = ",", skip_header = 1)
    data = data[:, 1:-1]
    return data


2.1.2. Normalizar as features no intervalo [0, 1].

In [1319]:
def normalize_features(data):
    max_values = data.max(axis = 0)
    min_values = data.min(axis = 0)
    return (data-min_values)/(max_values - min_values)

2.1.3. Criar e gravar em ficheiro um array numpy com as features extraídas (linhas = músicas; colunas = valores das features).

In [1320]:
def save_to_file(path, data):
    np.savetxt(path, data, delimiter=",", fmt="%.6f")

2.2. Extrair features da framework librosa. <br>

2.2.1. Para os 900 ficheiros da BD, extrair as seguintes features (sugestão: guardar
todas as músicas na mesma pasta):
- Features Espectrais: mfcc, spectral centroid, spectral bandwidth, spectral
contrast, spectral flatness e spectral rolloff.
- Features Temporais: F0, rms e zero crossing rate.
- Outras features: tempo.
- Utilize os parâmetros por omissão do librosa (sr = 22050 Hz, mono, window
length = frame length = 92.88 ms e hop length = 23.22 ms).
- Guarde as features num array numpy 2D, com número de linhas = número de
músicas e número de colunas = número de features

In [1321]:
def extract_music_features(music):
    path = "Music/"
    
    data, sampling_rate = librosa.load(path + music)

    mfcc = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc = 13)
    spectral_centroid = librosa.feature.spectral_centroid(y=data, sr=sampling_rate)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=data, sr=sampling_rate)
    spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=sampling_rate)
    spectral_flatness = librosa.feature.spectral_flatness(y=data)
    spectral_rolloff = librosa.feature.spectral_rolloff(y = data, sr = sampling_rate)
    f0 = librosa.yin(y=data, fmin=20, fmax=11025)
    f0[f0 == 11025] = 0
    rms = librosa.feature.rms(y=data)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=data)
    tempo = librosa.beat.tempo(y=data, sr=sampling_rate)
        
    return mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff, f0, rms, zero_crossing_rate, tempo

2.2.2. Calcular as 7 estatísticas típicas sobre as features anteriores: média, desvio
padrão, assimetria (skewness), curtose (kurtosis), mediana, máximo e mínimo.
Para o efeito, utilizar a biblioteca scipy.stats (e.g., scipy.stats.skew).

In [1322]:
def calculate_statistics(feature, axis_):
    feat_mean = feature.mean(axis = axis_)
    feat_std = feature.std(axis = axis_)
    feat_skewness = stats.skew(feature, axis = axis_)
    feat_kurtosis = stats.kurtosis(feature, axis = axis_)
    feat_median = np.median(feature, axis = axis_)
    feat_max = feature.max(axis = axis_)
    feat_min = feature.min(axis = axis_)
    
    if(axis_ == 1):
        result = np.array([])
        for i in range(len(feat_mean)):
            result = np.hstack((result, feat_mean[i], feat_std[i], feat_skewness[i], feat_kurtosis[i], feat_median[i], feat_max[i], feat_min[i]))
        return result
    else:
        # f0
        return np.hstack(([feat_mean], [feat_std], [feat_skewness], [feat_kurtosis], [feat_median], [feat_max], [feat_min]))
    

#### **3.Implementação de métricas de similaridade.**

3.1. Desenvolver o código Python/numpy para calcular as seguintes métricas de similaridade:

3.1.1. Distância Euclidiana 

In [1323]:
def euclidean_distance(a,b):
    distance = np.linalg.norm(a-b)
    return distance, distance

3.1.2. Manhattan distance

In [1324]:
def manhattan_distance(a, b):
    distance = sp.spatial.distance.cityblock(a, b)
    return distance, distance

3.1.3. Cosine distance

In [1325]:
def cosine_distance(a, b):
    distance = spatial.distance.cosine(a, b)
    return distance, distance


In [1326]:
def calculate_distances(features, feature_type):
    data = np.empty((len(features), len(features)))
 
    for i in range (0, len(features)):
        for j in range (i, len(features)):
            if i == j:
                data[i][j] = 0
            elif feature_type == "Euclidean":
                data[i][j], data[j][i] = euclidean_distance(features[i], features[j])
            elif feature_type == "Manhattan":
                data[i][j], data[j][i] = manhattan_distance(features[i], features[j])
            elif feature_type == "Cosine":
                data[i][j], data[j][i] = cosine_distance(features[i], features[j])

    return data

3.2. Criar e gravar em ficheiro 6 (ou 9, se bonificação) matrizes de similaridade (900x900), uma para cada conjunto de features e métrica de distância utilizada.

In [1327]:
def receive_distances(filename, data, type):
    if not os.path.exists(filename):
        data[data != data] = 0
        data_output = calculate_distances(data, type)
        save_to_file(filename, data_output)
        return data_output
    else:
        return np.genfromtxt(filename, delimiter = ",")   

3.3. Criar os 6 (ou 9) rankings de similaridade (para as 4 queries fornecidos). Considere apenas recomendações de 20 músicas.

In [1328]:
def print_20_best_rankings(data, indexes):
    rankings = np.array([])
    for i in range(1,21):
        print(f"\t{i}º - {data[indexes[i]]}")
        rankings = np.concatenate((rankings, data[indexes[i]]), axis = None)
    return rankings

3.4. Apresentar, comparar e discutir os resultados.

#### **4. Avaliação**

4.1. Avaliação objectiva.

4.1.1. Para cada uma das 4 queries, obter o ranking das 20 músicas recomendadas com base na correspondência com os metadados seguintes: artista, género, quadrante e emoção. Por cada item coincidente, adicionar um ponto à qualidade da música alvo, e.g., se tanto a música de referência como o alvo tiverem género = jazz e emoção = happy, a qualidade do alvo será 2.

In [1329]:
def metadata_extraction():
    data = np.genfromtxt("Dataset/panda_dataset_taffc_metadata.csv", delimiter = ",", dtype = str, skip_header = 1)

    dic = {}
    for line in data:
        genres = line[11][1:-1].split("; ")
        emotions = line[9][1:-1].split("; ")
        
        dic[line[0][1:-1]] = [line[1][1:-1], genres, line[3][1:-1], emotions]
    return dic

In [1330]:
def calculate_music_quality(queries, dic, path):
    qualities = {}
    queries_qualities = np.empty((4, 900))
    
    i = 0
    for querie in queries:
        querie = querie.replace(".mp3", "")
                
        j = 0
        for key, value in dic.items():  
            sum_ = 0

            if (key == querie):
                sum_ = -1
            else:
                # Artist
                sum_ += (dic[querie][0] == value[0])
            
                # Genre
                sum_ += len(list(set(dic[querie][1]).intersection(value[1])))
                
                # Quadrant
                sum_ += (dic[querie][2] == value[2])
                
                # Emotion
                sum_ += len(list(set(dic[querie][3]).intersection(value[3])))
                
            queries_qualities[i][j] = sum_
            j += 1

        qualities[querie] = np.argsort(queries_qualities[i])[::-1]
        i += 1
    
    save_to_file(path, queries_qualities)
    return qualities

4.1.2. Criar e gravar a matriz de similaridade baseada em contexto (i.e., nos metadados).

In [1331]:
def print_20_quality_rankings(music_folder, qualities_dic):
    array = np.empty((20), dtype = str)
    
    print("\n--------------- Exercice 4.1.2. ------------------")
    for key, index in qualities_dic.items():
        print(f"Ranking best 20 songs for {key}")
        
        a = np.array([])
        for i in range(20):
            print(f"{i+1}º - {music_folder[index[i]]}")  
            a = np.concatenate((a, [music_folder[index[i]]]))          
        array = np.vstack((array, a))
        print()
    return array[1:]

4.1.3. Para cada um dos rankings determinados em 3.3, calcular a métrica precision, assumindo como relevantes as músicas devolvidas em 4.1.1 (metadados).

In [1332]:
def calculate_precision(queries_folder, rankings_distance, rankings_metadata):
    print("--------------- Exercice 4.1.3. ------------------")
    for i in range(len(rankings_metadata)):
        precision_euclidean_data = len(list(set(rankings_metadata[i]).intersection(rankings_distance[i*6]))) / 20
        precision_euclidean_librosa = len(list(set(rankings_metadata[i]).intersection(rankings_distance[i*6 + 1]))) / 20
        precision_manhattan_data = len(list(set(rankings_metadata[i]).intersection(rankings_distance[i*6 + 2]))) / 20
        precision_manhattan_librosa = len(list(set(rankings_metadata[i]).intersection(rankings_distance[i*6 + 3]))) / 20
        precision_cosine_data = len(list(set(rankings_metadata[i]).intersection(rankings_distance[i*6 + 4]))) / 20
        precision_cosine_librosa = len(list(set(rankings_metadata[i]).intersection(rankings_distance[i*6 + 5]))) / 20
        
        print(f"--- PRECISIONS ({queries_folder[i]}) ---")
        print(f"Euclidean data: {precision_euclidean_data}")
        print(f"Euclidean librosa: {precision_euclidean_librosa}")
        print(f"Manhattan data: {precision_manhattan_data}")
        print(f"Manhattan librosa: {precision_manhattan_librosa}")
        print(f"Cosine data: {precision_cosine_data}")
        print(f"Cosine librosa: {precision_cosine_librosa}")
        print("----------------------------\n")

4.1.4. Apresentar, comparar e discutir os resultados.

#### **4.2. Avaliação subjectiva**

#### 4.2.1. Para cada uma das 4 queries, conjunto de 100 features e distância do coseno, avaliar a qualidade de cada uma das 20 recomendações, com base na seguinte escala de Likert ": 1 – Muito Má; 2 – Má; 3 – Aceitável; 4 – Boa; 5 – Muito Boa (para as 4 queries). Cada elemento do grupo deverá efectuar individualmente a avaliação da recomendação.

a) Calcular a média e o desvio-padrão de todos os membros por query, assim como a média e o desvio-padrão global para as 4 queries.<br>
b) Definindo um score mínimo de 2.5 para “recomendação relevante”, calcular a precision resultante.

| top100\_cos: MT0000202045.mp3 | Alexy        | Sofia        | Tatiana      | Mean         | Std          | Count |
| ----------------------------- | ------------ | ------------ | ------------ | ------------ | ------------ | ----- |
| 1º - MT0003106472.mp3         | 2            | 3            | 3            | 2,666666667  | 0,5773502692 | 1     |
| 2º - MT0010736208.mp3         | 2            | 1            | 2            | 1,666666667  | 0,5773502692 | 0     |
| 3º - MT0014703649.mp3         | 2            | 1            | 2            | 1,666666667  | 0,5773502692 | 0     |
| 4º - MT0005409948.mp3         | 2            | 1            | 2            | 1,666666667  | 0,5773502692 | 0     |
| 5º - MT0014615863.mp3         | 2            | 1            | 2            | 1,666666667  | 0,5773502692 | 0     |
| 6º - MT0004882280.mp3         | 1            | 1            | 1            | 1            | 0            | 0     |
| 7º - MT0013612461.mp3         | 1            | 1            | 1            | 1            | 0            | 0     |
| 8º - MT0003724610.mp3         | 2            | 2            | 2            | 2            | 0            | 0     |
| 9º - MT0002372242.mp3         | 2            | 1            | 2            | 1,666666667  | 0,5773502692 | 0     |
| 10º - MT0004032071.mp3        | 2            | 1            | 1            | 1,333333333  | 0,5773502692 | 0     |
| 11º - MT0010615428.mp3        | 1            | 1            | 1            | 1            | 0            | 0     |
| 12º - MT0040033011.mp3        | 2            | 2            | 2            | 2            | 0            | 0     |
| 13º - MT0007535042.mp3        | 1            | 1            | 2            | 1,333333333  | 0,5773502692 | 0     |
| 14º - MT0004293364.mp3        | 2            | 1            | 1            | 1,333333333  | 0,5773502692 | 0     |
| 15º - MT0009800907.mp3        | 1            | 1            | 1            | 1            | 0            | 0     |
| 16º - MT0004028719.mp3        | 3            | 3            | 2            | 2,666666667  | 0,5773502692 | 1     |
| 17º - MT0030036616.mp3        | 3            | 2            | 2            | 2,333333333  | 0,5773502692 | 0     |
| 18º - MT0004085907.mp3        | 1            | 1            | 1            | 1            | 0            | 0     |
| 19º - MT0010624346.mp3        | 2            | 1            | 1            | 1,333333333  | 0,5773502692 | 0     |
| 20º - MT0014794891.mp3        | 1            | 3            | 2            | 2            | 1            | 0     |
| Mean                          | 1,75         | 1,45         | 1,65         | 1,616666667  | 0,3964101615 | 0,1   |
| Std                           | 0,6386663737 | 0,7591546545 | 0,5871429486 | 0,5328396838 | 0,3126440449 |       |
| Precision                     | 10,00%       |              |              |              |              |

<br><br>

| top100\_cos: MT0000379144.mp3 | Alexy       | Sofia       | Tatiana     | Mean        | Std          | Count |
| ----------------------------- | ----------- | ----------- | ----------- | ----------- | ------------ | ----- |
| 1º - MT0009010830.mp3         | 1           | 1           | 1           | 1           | 0            | 0     |
| 2º - MT0003778826.mp3         | 1           | 1           | 1           | 1           | 0            | 0     |
| 3º - MT0015005100.mp3         | 2           | 2           | 2           | 2           | 0            | 0     |
| 4º - MT0028627699.mp3         | 3           | 2           | 2           | 2,333333333 | 0,5773502692 | 0     |
| 5º - MT0012124855.mp3         | 1           | 1           | 1           | 1           | 0            | 0     |
| 6º - MT0027035970.mp3         | 4           | 5           | 4           | 4,333333333 | 0,5773502692 | 1     |
| 7º - MT0000044741.mp3         | 4           | 4           | 4           | 4           | 0            | 1     |
| 8º - MT0003794106.mp3         | 1           | 1           | 1           | 1           | 0            | 0     |
| 9º - MT0005478759.mp3         | 1           | 1           | 1           | 1           | 0            | 0     |
| 10º - MT0002262181.mp3        | 2           | 1           | 1           | 1,333333333 | 0,5773502692 | 0     |
| 11º - MT0031996897.mp3        | 1           | 3           | 2           | 2           | 1            | 0     |
| 12º - MT0000992846.mp3        | 1           | 1           | 1           | 1           | 0            | 0     |
| 13º - MT0001376988.mp3        | 1           | 1           | 1           | 1           | 0            | 0     |
| 14º - MT0009208842.mp3        | 1           | 1           | 1           | 1           | 0            | 0     |
| 15º - MT0010465830.mp3        | 3           | 2           | 2           | 2,333333333 | 0,5773502692 | 0     |
| 16º - MT0005737276.mp3        | 4           | 3           | 4           | 3,666666667 | 0,5773502692 | 1     |
| 17º - MT0018031959.mp3        | 1           | 1           | 1           | 1           | 0            | 0     |
| 18º - MT0033958450.mp3        | 4           | 4           | 4           | 4           | 0            | 1     |
| 19º - MT0008222676.mp3        | 2           | 2           | 3           | 2,333333333 | 0,5773502692 | 0     |
| 20º - MT0012041920.mp3        | 3           | 3           | 3           | 3           | 0            | 1     |
| Mean                          | 2,05        | 2           | 2           | 2,016666667 | 0,2232050808 | 0,25  |
| Std                           | 1,234376041 | 1,256561725 | 1,213953957 | 1,191956473 | 0,3247338861 |       |
| Precision                     | 25,00%      |             |             |             |              |

<br><br>

| top100\_cos: MT0000414517.mp3 | Alexy       | Sofia        | Tatiana      | Mean         | Std          | Count |
| ----------------------------- | ----------- | ------------ | ------------ | ------------ | ------------ | ----- |
| 1º - MT0003900455.mp3         | 3           | 1            | 2            | 2            | 1            | 0     |
| 2º - MT0009897495.mp3         | 2           | 1            | 1            | 1,333333333  | 0,5773502692 | 0     |
| 3º - MT0000203193.mp3         | 1           | 1            | 1            | 1            | 0            | 0     |
| 4º - MT0000901959.mp3         | 1           | 1            | 1            | 1            | 0            | 0     |
| 5º - MT0009521580.mp3         | 1           | 1            | 1            | 1            | 0            | 0     |
| 6º - MT0013955066.mp3         | 2           | 3            | 2            | 2,333333333  | 0,5773502692 | 0     |
| 7º - MT0000888329.mp3         | 3           | 2            | 3            | 2,666666667  | 0,5773502692 | 1     |
| 8º - MT0004428604.mp3         | 2           | 1            | 2            | 1,666666667  | 0,5773502692 | 0     |
| 9º - MT0018651126.mp3         | 2           | 1            | 2            | 1,666666667  | 0,5773502692 | 0     |
| 10º - MT0009346128.mp3        | 1           | 1            | 1            | 1            | 0            | 0     |
| 11º - MT0002379222.mp3        | 1           | 1            | 1            | 1            | 0            | 0     |
| 12º - MT0034186195.mp3        | 3           | 2            | 2            | 2,333333333  | 0,5773502692 | 0     |
| 13º - MT0017667847.mp3        | 1           | 1            | 1            | 1            | 0            | 0     |
| 14º - MT0000218346.mp3        | 1           | 1            | 1            | 1            | 0            | 0     |
| 15º - MT0000040632.mp3        | 2           | 3            | 2            | 2,333333333  | 0,5773502692 | 0     |
| 16º - MT0002634024.mp3        | 3           | 1            | 2            | 2            | 1            | 0     |
| 17º - MT0012331779.mp3        | 5           | 4            | 4            | 4,333333333  | 0,5773502692 | 1     |
| 18º - MT0001703346.mp3        | 2           | 2            | 3            | 2,333333333  | 0,5773502692 | 0     |
| 19º - MT0027835071.mp3        | 3           | 2            | 2            | 2,333333333  | 0,5773502692 | 0     |
| 20º - MT0009991160.mp3        | 1           | 1            | 1            | 1            | 0            | 0     |
| Mean                          | 2           | 1,55         | 1,75         | 1,766666667  | 0,3886751346 | 0,1   |
| Std                           | 1,076055174 | 0,8870412083 | 0,8506963092 | 0,8590760711 | 0,3488300844 |       |
| Precision                     | 10,00%      |              |              |              |              |

<br><br>

| top100\_cos: MT0000956340.mp3 | Alexy       | Sofia       | Tatiana     | Mean        | Std          | Count |
| ----------------------------- | ----------- | ----------- | ----------- | ----------- | ------------ | ----- |
| 1º - MT0003106472.mp3         | 5           | 4           | 4           | 4,333333333 | 0,5773502692 | 1     |
| 2º - MT0010736208.mp3         | 5           | 3           | 4           | 4           | 1            | 1     |
| 3º - MT0014703649.mp3         | 5           | 3           | 4           | 4           | 1            | 1     |
| 4º - MT0005409948.mp3         | 4           | 4           | 4           | 4           | 0            | 1     |
| 5º - MT0014615863.mp3         | 4           | 5           | 5           | 4,666666667 | 0,5773502692 | 1     |
| 6º - MT0004882280.mp3         | 2           | 2           | 3           | 2,333333333 | 0,5773502692 | 0     |
| 7º - MT0013612461.mp3         | 4           | 4           | 4           | 4           | 0            | 1     |
| 8º - MT0003724610.mp3         | 3           | 4           | 3           | 3,333333333 | 0,5773502692 | 1     |
| 9º - MT0002372242.mp3         | 4           | 5           | 4           | 4,333333333 | 0,5773502692 | 1     |
| 10º - MT0004032071.mp3        | 1           | 1           | 1           | 1           | 0            | 0     |
| 11º - MT0010615428.mp3        | 4           | 4           | 4           | 4           | 0            | 1     |
| 12º - MT0040033011.mp3        | 4           | 3           | 4           | 3,666666667 | 0,5773502692 | 1     |
| 13º - MT0007535042.mp3        | 1           | 1           | 1           | 1           | 0            | 0     |
| 14º - MT0004293364.mp3        | 4           | 5           | 4           | 4,333333333 | 0,5773502692 | 1     |
| 15º - MT0009800907.mp3        | 2           | 2           | 2           | 2           | 0            | 0     |
| 16º - MT0004028719.mp3        | 1           | 1           | 1           | 1           | 0            | 0     |
| 17º - MT0030036616.mp3        | 3           | 3           | 2           | 2,666666667 | 0,5773502692 | 1     |
| 18º - MT0004085907.mp3        | 2           | 1           | 1           | 1,333333333 | 0,5773502692 | 0     |
| 19º - MT0010624346.mp3        | 3           | 4           | 4           | 3,666666667 | 0,5773502692 | 1     |
| 20º - MT0014794891.mp3        | 3           | 4           | 3           | 3,333333333 | 0,5773502692 | 1     |
| Mean                          | 3,2         | 3,15        | 3,1         | 3,15        | 0,4175426481 | 0,7   |
| Std                           | 1,321880638 | 1,386969434 | 1,293709477 | 1,258770402 | 0,3387150182 |       |
| Precision                     | 70,00%      |             |             |             |              |

##### 4.2.2. Para cada uma das 4 queries e similaridade com base nos metadados, avaliar a qualidade de cada uma das 20 recomendações, com base na escala de Likert anterior. Cada elemento do grupo deverá efectuar individualmente a avaliação da recomendação.

a) Calcular a média e o desvio-padrão de todos os membros por query, assim como a média e o desvio-padrão global para as 4 queries.

| metadata: MT0000202045.mp3 | Alexy       | Sofia       | Tatiana     | Mean        | Std          | Count |
| -------------------------- | ----------- | ----------- | ----------- | ----------- | ------------ | ----- |
| 1º - MT0014475915.mp3      | 2           | 4           | 3           | 3           | 1            | 1     |
| 2º - MT0012862507.mp3      | 1           | 1           | 1           | 1           | 0            | 0     |
| 3º - MT0000888329.mp3      | 5           | 5           | 4           | 4,666666667 | 0,5773502692 | 1     |
| 4º - MT0007556029.mp3      | 3           | 3           | 3           | 3           | 0            | 1     |
| 5º - MT0031898123.mp3      | 2           | 1           | 1           | 1,333333333 | 0,5773502692 | 0     |
| 6º - MT0004867564.mp3      | 1           | 1           | 1           | 1           | 0            | 0     |
| 7º - MT0001494812.mp3      | 2           | 2           | 1           | 1,666666667 | 0,5773502692 | 0     |
| 8º - MT0003022328.mp3      | 2           | 3           | 2           | 2,333333333 | 0,5773502692 | 0     |
| 9º - MT0011922905.mp3      | 3           | 3           | 2           | 2,666666667 | 0,5773502692 | 1     |
| 10º - MT0030369896.mp3     | 2           | 2           | 2           | 2           | 0            | 0     |
| 11º - MT0007453719.mp3     | 3           | 2           | 2           | 2,333333333 | 0,5773502692 | 0     |
| 12º - MT0034186620.mp3     | 4           | 3           | 4           | 3,666666667 | 0,5773502692 | 1     |
| 13º - MT0004850690.mp3     | 2           | 2           | 2           | 2           | 0            | 0     |
| 14º - MT0011938737.mp3     | 1           | 2           | 3           | 2           | 1            | 0     |
| 15º - MT0034577404.mp3     | 3           | 1           | 2           | 2           | 1            | 0     |
| 16º - MT0003025046.mp3     | 2           | 3           | 3           | 2,666666667 | 0,5773502692 | 1     |
| 17º - MT0005285696.mp3     | 1           | 2           | 1           | 1,333333333 | 0,5773502692 | 0     |
| 18º - MT0002846256.mp3     | 1           | 1           | 1           | 1           | 0            | 0     |
| 19º - MT0001058887.mp3     | 5           | 3           | 4           | 4           | 1            | 1     |
| 20º - MT0007766156.mp3     | 2           | 2           | 2           | 2           | 0            | 0     |
| Mean                       | 2,35        | 2,3         | 2,2         | 2,283333333 | 0,4598076211 | 0,35  |
| Std                        | 1,225818738 | 1,080935268 | 1,056309365 | 1,010326796 | 0,3819299351 |       |
| Precision                  | 35,00%      |             |             |             |              |

<br><br>

| metadata: MT0000379144.mp3 | Alexy        | Sofia       | Tatiana | Mean         | Std          | Count |
| -------------------------- | ------------ | ----------- | ------- | ------------ | ------------ | ----- |
| 1º - MT0031951901.mp3      | 4            | 4           | 4       | 4            | 0            | 1     |
| 2º - MT0014584473.mp3      | 4            | 4           | 4       | 4            | 0            | 1     |
| 3º - MT0013080259.mp3      | 4            | 3           | 4       | 3,666666667  | 0,5773502692 | 1     |
| 4º - MT0013416300.mp3      | 4            | 3           | 3       | 3,333333333  | 0,5773502692 | 1     |
| 5º - MT0011032905.mp3      | 4            | 3           | 3       | 3,333333333  | 0,5773502692 | 1     |
| 6º - MT0005157391.mp3      | 3            | 4           | 3       | 3,333333333  | 0,5773502692 | 1     |
| 7º - MT0005253065.mp3      | 4            | 3           | 4       | 3,666666667  | 0,5773502692 | 1     |
| 8º - MT0008170600.mp3      | 5            | 5           | 4       | 4,666666667  | 0,5773502692 | 1     |
| 9º - MT0007652281.mp3      | 5            | 4           | 4       | 4,333333333  | 0,5773502692 | 1     |
| 10º - MT0007349999.mp3     | 3            | 4           | 3       | 3,333333333  | 0,5773502692 | 1     |
| 11º - MT0007338724.mp3     | 2            | 2           | 2       | 2            | 0            | 0     |
| 12º - MT0001526386.mp3     | 5            | 5           | 5       | 5            | 0            | 1     |
| 13º - MT0004287283.mp3     | 5            | 5           | 4       | 4,666666667  | 0,5773502692 | 1     |
| 14º - MT0005115042.mp3     | 5            | 5           | 4       | 4,666666667  | 0,5773502692 | 1     |
| 15º - MT0004131058.mp3     | 3            | 2           | 2       | 2,333333333  | 0,5773502692 | 0     |
| 16º - MT0001929641.mp3     | 3            | 3           | 4       | 3,333333333  | 0,5773502692 | 1     |
| 17º - MT0001934726.mp3     | 4            | 3           | 3       | 3,333333333  | 0,5773502692 | 1     |
| 18º - MT0003863509.mp3     | 2            | 2           | 1       | 1,666666667  | 0,5773502692 | 0     |
| 19º - MT0029877658.mp3     | 5            | 5           | 5       | 5            | 0            | 1     |
| 20º - MT0003114552.mp3     | 4            | 5           | 4       | 4,333333333  | 0,5773502692 | 1     |
| Mean                       | 3,9          | 3,7         | 3,5     | 3,7          | 0,4330127019 | 0,85  |
| Std                        | 0,9679060415 | 1,080935268 | 1       | 0,9421885683 | 0,256494588  |       |
| Precision                  | 85,00%       |             |         |              |              |

<br><br>


| metadata: MT0000414517.mp3 | Alexy      | Sofia       | Tatiana     | Mean        | Std          | Count |
| -------------------------- | ---------- | ----------- | ----------- | ----------- | ------------ | ----- |
| 1º - MT0010489498.mp3      | 5          | 3           | 4           | 4           | 1            | 1     |
| 2º - MT0000040632.mp3      | 5          | 5           | 5           | 5           | 0            | 1     |
| 3º - MT0010487769.mp3      | 5          | 4           | 5           | 4,666666667 | 0,5773502692 | 1     |
| 4º - MT0012331779.mp3      | 5          | 5           | 5           | 5           | 0            | 1     |
| 5º - MT0027048677.mp3      | 5          | 5           | 5           | 5           | 0            | 1     |
| 6º - MT0033397838.mp3      | 4          | 5           | 4           | 4,333333333 | 0,5773502692 | 1     |
| 7º - MT0002222957.mp3      | 5          | 5           | 5           | 5           | 0            | 1     |
| 8º - MT0003949060.mp3      | 3          | 5           | 4           | 4           | 1            | 1     |
| 9º - MT0007840454.mp3      | 2          | 2           | 2           | 2           | 0            | 0     |
| 10º - MT0008222676.mp3     | 2          | 4           | 4           | 3,333333333 | 1,154700538  | 1     |
| 11º - MT0010900969.mp3     | 3          | 1           | 2           | 2           | 1            | 0     |
| 12º - MT0003286463.mp3     | 2          | 1           | 2           | 1,666666667 | 0,5773502692 | 0     |
| 13º - MT0003243311.mp3     | 4          | 3           | 2           | 3           | 1            | 1     |
| 14º - MT0015005100.mp3     | 1          | 1           | 1           | 1           | 0            | 0     |
| 15º - MT0002033629.mp3     | 5          | 4           | 4           | 4,333333333 | 0,5773502692 | 1     |
| 16º - MT0004141823.mp3     | 2          | 2           | 2           | 2           | 0            | 0     |
| 17º - MT0027159893.mp3     | 3          | 2           | 2           | 2,333333333 | 0,5773502692 | 0     |
| 18º - MT0009729892.mp3     | 2          | 2           | 2           | 2           | 0            | 0     |
| 19º - MT0004751933.mp3     | 3          | 4           | 3           | 3,333333333 | 0,5773502692 | 1     |
| 20º - MT0006315323.mp3     | 3          | 3           | 3           | 3           | 0            | 1     |
| Mean                       | 3,45       | 3,3         | 3,3         | 3,35        | 0,4309401077 | 0,65  |
| Std                        | 1,35627198 | 1,490319641 | 1,341640786 | 1,304401007 | 0,4364417008 |       |
| Precision                  | 65,00%     |             |             |             |              |

<br><br>

| metadata: MT0000956340.mp3 | Alexy       | Sofia       | Tatiana     | Mean        | Std          | Count |
| -------------------------- | ----------- | ----------- | ----------- | ----------- | ------------ | ----- |
| 1º - MT0016743722.mp3      | 5           | 4           | 4           | 4,333333333 | 0,5773502692 | 1     |
| 2º - MT0014615863.mp3      | 5           | 5           | 5           | 5           | 0            | 1     |
| 3º - MT0013885218.mp3      | 4           | 4           | 4           | 4           | 0            | 1     |
| 4º - MT0004293364.mp3      | 5           | 4           | 5           | 4,666666667 | 0,5773502692 | 1     |
| 5º - MT0033097471.mp3      | 4           | 5           | 5           | 4,666666667 | 0,5773502692 | 1     |
| 6º - MT0014845647.mp3      | 2           | 2           | 2           | 2           | 0            | 0     |
| 7º - MT0012001409.mp3      | 3           | 3           | 2           | 2,666666667 | 0,5773502692 | 1     |
| 8º - MT0001927746.mp3      | 1           | 1           | 1           | 1           | 0            | 0     |
| 9º - MT0001891229.mp3      | 2           | 1           | 1           | 1,333333333 | 0,5773502692 | 0     |
| 10º - MT0010897525.mp3     | 1           | 1           | 1           | 1           | 0            | 0     |
| 11º - MT0029099688.mp3     | 1           | 1           | 1           | 1           | 0            | 0     |
| 12º - MT0012914763.mp3     | 1           | 1           | 1           | 1           | 0            | 0     |
| 13º - MT0004459450.mp3     | 1           | 1           | 1           | 1           | 0            | 0     |
| 14º - MT0009217411.mp3     | 4           | 3           | 4           | 3,666666667 | 0,5773502692 | 1     |
| 15º - MT0005897799.mp3     | 4           | 3           | 4           | 3,666666667 | 0,5773502692 | 1     |
| 16º - MT0032957418.mp3     | 5           | 5           | 5           | 5           | 0            | 1     |
| 17º - MT0014817509.mp3     | 2           | 2           | 2           | 2           | 0            | 0     |
| 18º - MT0015541501.mp3     | 1           | 2           | 2           | 1,666666667 | 0,5773502692 | 0     |
| 19º - MT0002372242.mp3     | 4           | 2           | 3           | 3           | 1            | 1     |
| 20º - MT0015962332.mp3     | 4           | 2           | 3           | 3           | 1            | 1     |
| Mean                       | 2,95        | 2,6         | 2,8         | 2,783333333 | 0,3309401077 | 0,55  |
| Std                        | 1,605090586 | 1,465390194 | 1,576137851 | 1,503504678 | 0,3610102469 |       |
| Precision                  | 55,00%      |             |             |             |              |

4.2.3. Apresentar, comparar e discutir os resultados.

#### **Main function**

In [1333]:
if __name__ == "__main__" :
    warnings.filterwarnings("ignore")
    
    saved_features = "top100_features_normalized.csv"
    librosa_features = "librosa_features_normalized.csv"
    music_folder = os.listdir("Music")
    queries_folder = os.listdir("Queries")
    
    
    # Ex 2.1
    if os.path.exists(saved_features):
        data_normalized = np.genfromtxt(saved_features, delimiter = ",")
    else:
        data_extracted = extract_features()
        data_normalized = normalize_features(data_extracted)
        save_to_file(saved_features, data_normalized)
    
    
    # Ex 2.2
    if os.path.exists(librosa_features):
        librosa_data_normalized = np.genfromtxt(librosa_features, delimiter = ",")
    else:
        statistics = np.empty((0, 190))
        
        iteration = 1
        for music in music_folder:
            mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff, f0, rms, zero_crossing_rate, tempo = extract_music_features(music)
            
            mfcc_statistics = calculate_statistics(mfcc, 1)
            spectral_centroid_statistics = calculate_statistics(spectral_centroid, 1)
            spectral_bandwidth_statistics = calculate_statistics(spectral_bandwidth, 1)
            spectral_contrast_statistics = calculate_statistics(spectral_contrast, 1)
            spectral_flatness_statistics = calculate_statistics(spectral_flatness, 1)
            spectral_rolloff_statistics = calculate_statistics(spectral_rolloff, 1)
            f0_statistics = calculate_statistics(f0, 0)
            rms_statistics = calculate_statistics(rms, 1)
            zero_crossing_rate_statistics = calculate_statistics(zero_crossing_rate, 1)
            music_statistics = np.concatenate((mfcc_statistics.flatten(), 
                                                spectral_centroid_statistics,
                                                spectral_bandwidth_statistics,
                                                spectral_contrast_statistics.flatten(),
                                                spectral_flatness_statistics,
                                                spectral_rolloff_statistics,
                                                f0_statistics, 
                                                rms_statistics,
                                                zero_crossing_rate_statistics,
                                                tempo))
            statistics = np.vstack((statistics, music_statistics))
            iteration += 1 
            print(iteration)

        
        librosa_data_normalized = normalize_features(statistics)
        save_to_file(librosa_features, librosa_data_normalized)

    # Ex 3.1 e 3.2
    fich_names = ["data_euclidean_distance.csv", "librosa_euclidean_distance.csv","data_manhattan_distance.csv", "librosa_manhattan_distance.csv", "data_cosine_distance.csv", "librosa_cosine_distance.csv"]

    #       3.1.1
    data_euclidean_distance = receive_distances(fich_names[0], data_normalized, "Euclidean")
    librosa_euclidean_distance = receive_distances(fich_names[1], librosa_data_normalized, "Euclidean")
    
    #       3.1.2
    data_manhattan_distance = receive_distances(fich_names[2], data_normalized, "Manhattan")
    librosa_manhattan_distance = receive_distances(fich_names[3], librosa_data_normalized, "Manhattan")
    
    #       3.1.3
    data_cosine_distance = receive_distances(fich_names[4], data_normalized, "Cosine")
    librosa_cosine_distance = receive_distances(fich_names[5], librosa_data_normalized, "Cosine")


    # Ex 3.3
    
    rankings_distance = np.empty((20), dtype = str)
    print("--------------- Exercice 3.3. ------------------")
    for i in range(len(queries_folder)):
        print(f">>>>> Song {queries_folder[i]} <<<<<")
        print("Ranking with data euclidean distance")
        indice = music_folder.index(queries_folder[i])

        indexes = np.argsort(data_euclidean_distance[indice])
        rankings_distance = np.vstack((rankings_distance, print_20_best_rankings(music_folder, indexes)))
        
        print("\nRanking with librosa euclidean distance")
        indexes = np.argsort(librosa_euclidean_distance[indice])
        rankings_distance = np.vstack((rankings_distance, print_20_best_rankings(music_folder, indexes)))
        
        print("\nRanking with data manhattan distance")
        indexes = np.argsort(data_manhattan_distance[indice])
        rankings_distance = np.vstack((rankings_distance, print_20_best_rankings(music_folder, indexes)))
        
        print("\nRanking with librosa manhattan distance")
        indexes = np.argsort(librosa_manhattan_distance[indice])
        rankings_distance = np.vstack((rankings_distance, print_20_best_rankings(music_folder, indexes)))
        
        print("\nRanking with data cosine distance")
        indexes = np.argsort(data_cosine_distance[indice])
        rankings_distance = np.vstack((rankings_distance, print_20_best_rankings(music_folder, indexes)))
                
        print("\nRanking with librosa cosine distance")
        indexes = np.argsort(librosa_cosine_distance[indice])
        rankings_distance = np.vstack((rankings_distance, print_20_best_rankings(music_folder, indexes)))
        print()

    rankings_distance = rankings_distance[1:]
    #print(rankings_distance)

    # Ex 4.1.1
    qualities_dic = {}
    path = "similarity_matrix.csv"
    if os.path.exists(path):
        data = np.genfromtxt(path, delimiter = ",")
        
        for i in range(len(queries_folder)):
            qualities_dic[queries_folder[i]] = np.argsort(data[i])[::-1]
            
    else:
        dic = metadata_extraction()
        qualities_dic = calculate_music_quality(queries_folder, dic, path)
        
    # Ex 4.1.2
    rankings_metadata = print_20_quality_rankings(music_folder, qualities_dic)

    # Ex 4.1.3
    calculate_precision(queries_folder, rankings_distance, rankings_metadata)
    
        

--------------- Exercice 3.3. ------------------
>>>>> Song MT0000202045.mp3 <<<<<
Ranking with data euclidean distance
	1º - MT0033841575.mp3
	2º - MT0027002641.mp3
	3º - MT0030487841.mp3
	4º - MT0008575372.mp3
	5º - MT0014576739.mp3
	6º - MT0030422114.mp3
	7º - MT0003390733.mp3
	8º - MT0027835071.mp3
	9º - MT0009188643.mp3
	10º - MT0010617945.mp3
	11º - MT0011145388.mp3
	12º - MT0009213083.mp3
	13º - MT0005331755.mp3
	14º - MT0002233402.mp3
	15º - MT0026727455.mp3
	16º - MT0005265641.mp3
	17º - MT0010344415.mp3
	18º - MT0000711493.mp3
	19º - MT0004428604.mp3
	20º - MT0018029465.mp3

Ranking with librosa euclidean distance
	1º - MT0005129157.mp3
	2º - MT0011899302.mp3
	3º - MT0012001409.mp3
	4º - MT0002233402.mp3
	5º - MT0007043504.mp3
	6º - MT0007799677.mp3
	7º - MT0004428604.mp3
	8º - MT0002161109.mp3
	9º - MT0011975274.mp3
	10º - MT0010624346.mp3
	11º - MT0009217411.mp3
	12º - MT0000092267.mp3
	13º - MT0007766156.mp3
	14º - MT0001676671.mp3
	15º - MT0005270263.mp3
	16º - MT00037874